In [189]:
# Importing libraries
import requests
import json
import polars as pl
from datetime import datetime

#code = '73bbdc709c3469a77fe5d6e36b37979e66537d05'
access_token = '87bfd70f523883ac4dc25143a2e3616755579875'
expires_at = '1746651070'
client_id = '156884'
client_secret = 'e10ddfed77d524fdf82e22fd682e4312b8d246f1'

In [ ]:
initial_auth_url = 'https://www.strava.com/api/v3/oauth/token'

payload = {'client_id': client_id, 
           'client_secret': client_secret,
           #'code': code,
           'grant_type': 'authorization_code'}

headers = {'Authorization' : f'Bearer {access_token}'}

r = requests.post(url = initial_auth_url, params=payload)
r = r.json()

In [190]:
r

{'token_type': 'Bearer',
 'expires_at': 1746649838,
 'expires_in': 15756,
 'refresh_token': 'f619a203f02a085497b94ec42b7ddac02b7540e3',
 'access_token': '9f5fefd745f35e47bc653a3b04cb8081427d0234',
 'athlete': {'id': 6996379,
  'username': 'anish_joni',
  'resource_state': 2,
  'firstname': 'Anish',
  'lastname': 'Joni',
  'bio': '',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'India',
  'sex': 'M',
  'premium': False,
  'summit': False,
  'created_at': '2014-11-05T01:05:56Z',
  'updated_at': '2025-04-25T13:09:53Z',
  'badge_type_id': 0,
  'weight': 79.4,
  'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/6996379/2109948/2/medium.jpg',
  'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/6996379/2109948/2/large.jpg',
  'friend': None,
  'follower': None}}

In [191]:
# First time auth details
tokens = {
    'access_token': f'{access_token}',
    'expires_at': f'{expires_at}',
    'client_id': f'{client_id}',
    'client_secret': f'{client_secret}'
}

In [167]:
# Utility functions
def load_tokens(path="../data/tokens.json"):
    with open(path, "r") as f:
       return json.load(f)

def save_tokens(tokens, path="../data/tokens.json"):
    with open(path, "w") as f:
        json.dump(tokens, f)

def is_token_expired(expires_at):
    if type(expires_at) == str:
        expires_at = int(expires_at)
    else:
        expires_at
    return datetime.now().timestamp() >= expires_at

def refresh_token_if_needed(client_id, client_secret):
    tokens = load_tokens()

    if is_token_expired(tokens['expires_at']):
        print('Access token expired. Refreshing...')

        url = "https://www.strava.com/oauth/token"
        payload = {
            'client_id': client_id,
            'client_secret': client_secret,
            'grant_type': 'refresh_token',
            'refresh_token': tokens['refresh_token']
        }

        r = requests.post(url=url, params=payload)

        if r.status_code == 200:
            new_tokens = r.json()
            tokens = {
                "access_token": new_tokens['access_token'],
                "refresh_token": new_tokens['refresh_token'],
                "expires_at": new_tokens['expires_at']
            }
            save_tokens(tokens)
        else:
            raise Exception(f"Token refresh failed: {r.status_code} - {r.text}")
    
    else:
        print('No token refresh needed.')
        
    return tokens['access_token']
        

In [192]:
# Persist initial token
save_tokens(tokens)

In [193]:
refresh_token_if_needed(client_id, client_secret)

No token refresh needed.


'87bfd70f523883ac4dc25143a2e3616755579875'

In [196]:
# Initial authentication
access_token = refresh_token_if_needed(client_id, client_secret)

# Request activities data from Strava API
activities_url = 'https://www.strava.com/api/v3/athlete/activities'
payload = {'client_id': client_id, 
           'client_secret': client_secret,
           'Authorization': f'Bearer {access_token}',
           'per_page' : 200,
             'page' : 2}

headers = {'Authorization': f'Bearer {access_token}'}
param = {'per_page' : 200, 'page' : 2}

activities = requests.get(url=activities_url, headers=headers).json()  

No token refresh needed.


In [200]:
access_token

'87bfd70f523883ac4dc25143a2e3616755579875'

In [ ]:

#

In [72]:
athlete_data = r.json()
athlete_data = pl.DataFrame([athlete_data])
athlete_data = athlete_data[['id','username', 'firstname', 'lastname', 'city', 'sex', 'weight']]


ColumnNotFoundError: "id" not found

In [93]:
print(expires_at)
print(datetime.now)
expires_at > datetime.now

2025-05-05 20:26:47
<built-in method now of type object at 0x00007FF84511FDA0>


TypeError: '>' not supported between instances of 'datetime.datetime' and 'builtin_function_or_method'

In [ ]:
athlete_data

id,username,firstname,lastname,city,sex,weight
i64,str,str,str,str,str,f64
6996379,"""anish_joni""","""Anish""","""Joni""","""Toronto""","""M""",79.4


In [216]:
# Request activities data from Strava API
activities_url = 'https://www.strava.com/api/v3/athlete '
headers = {'Authorization': f'Bearer {access_token}'}
param = {'per_page' : 200, 'page' : 2}

activities = requests.get(url=activities_url, headers=headers).json()  

In [217]:
access_token

'87bfd70f523883ac4dc25143a2e3616755579875'

In [218]:
activities

{'message': 'Record Not Found',
 'errors': [{'resource': 'resource', 'field': 'path', 'code': 'invalid'}]}

In [ ]:
activities.json()

{'message': 'Authorization Error',
 'errors': [{'resource': 'AccessToken',
   'field': 'activity:read_permission',
   'code': 'missing'}]}

In [ ]:
athlete_data.write_csv('strava_athlete.csv')